In [1]:
import sc3nb as scn

In [13]:
sc.exit()

NameError: name 'sc' is not defined

In [2]:
import sc3nb as scn

sc = scn.startup()


<IPython.core.display.Javascript object>

Found old process. Please exit sc3nb via sc.exit(). 
 Terminating psutil.Process(pid=23215, name='sclang', status='running', started='13:16:01') because none of the parents=[psutil.Process(pid=23212, name='SuperCollider', status='running', started='13:16:00'), psutil.Process(pid=1, name='launchd', status='running', started='2025-04-10 15:55:38'), psutil.Process(pid=0, name='kernel_task', status='running', started='2025-04-10 15:55:38')] are in allowed_parents=('scide', 'python', 'tox') More information can be found in the documentation.


Starting sclang process... [sclang | start reading ]
Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... [scsynth | start reading ]
Done.


In [5]:
sc.server.blip()

[scsynth | reached EOF ]
[sclang | reached EOF ]


In [8]:
%sc FreqScope(300); s.scope;

-> a Stethoscope


# Sound Synthesis with SuperCollider - crash course

**Learning Goals for today:**
* familiarize with fundamental audio-concepts in SC3
    * understand audio rate and control rate
    * understand how a graph of UGens is specified via a function in sclang
* learn the fundamental elements of a synthesizer
    * use the SC3 help system
    * continued in lecture
* get to know various interaction styles for controlling synths
    * via OSC messages, via Synth class, implicit function style, JIT ProxySpace, Patterns
* to be able to actively solve a synthesis task: e.g. synthesis of a water drop sound
* so that you understand the concepts and underlying 'mechanics'

## scSynth-Structure

efficient program for audio rendering 
* No SC code
* No Object
* No OOP

Only via Open Sound Control (OSC) controllable (via udp or tcp).

**Setup**
<img src="SC3-scsynth-intro-media/scsynth-overview.png" width="70%">

**Node**
* addressable object in a tree 

**Group** 	: Node
* List of nodes
* All Controls that are send to a group
    * are forwarded to the internal nodes

**Synth**		: Node
* tree structure
*  consists of unit generators (UGens)

**Synth Definition**
* contains the rules at hand of which a synth is created
* has a name (char*)

**Audio Bus**
* indexed by integers (0 up to …)
* usually 128
    * 0 .. numOutChannel-1
        * -> HW-Output
    * numOutChannel .. numOutChannel+numInChannel-1
        * -> HW-Input
	* beyond: private busses
	* In/Out.ar UGens can access (read / write)

**Control Bus**
* usually 1024
* In.kr/Out.kr UGens can access either read / write 

**Buffer**
* 32bit Float Arrays + header
* numbered
* non-RT instantiation and destruction
* Guaranteed access from the synth 
    * even during instantiation
* used for 
    * wave tables
	* sample buffers
	* delay lines
	* envelopes
	* …

**Unit Generator Definitions**
* Plugins
* binary code libs
* names matching scLang classes used in building SynthDefs

<img src="SC3-scsynth-intro-media/synth-nodes.png" width="70%"><br>

<img src="SC3-scsynth-intro-media/synth-example.png" width="50%"><br>

**Example (for the above shown figure as sclang code**:

In [9]:
%sc Clip.help 

SCDoc: Indexing help-files...
SCDoc: Indexed 1367 documents in 0.34 seconds
-> Clip


In [10]:
%sc s.makeGui

-> sc3nb_remote


In [11]:
%%scv
SynthDef("mySynth", { |freq=200, amp=0.1, sFreq=1| 
    Out.ar([0,1], SinOsc.ar(freq: freq, mul: amp) 
           * Clip.ar(Saw.ar(freq: sFreq, mul: 5)));
}).send(s)

-> a SynthDef


more transparently formatted:

In [12]:
%%scv
SynthDef(\mySynth, {|freq=200, amp=0.2, sFreq=1| 
    Out.ar(
        0,
        SinOsc.ar(freq, mul:amp)
        * 
        Clip.ar(Saw.ar(sFreq, 5))
        * 1!4  // 1.dup(4)
    );
}).send(s);

-> a SynthDef


In [13]:
%sc 1.dup(5) // --> [1,1,1,1,1] , also written as 1!5

-> [ 1, 1, 1, 1, 1 ]


this is how to use a synth

In [14]:
%sc x = Synth(\mySynth) // creates an instance 

-> Synth('mySynth' : 1006)


In [15]:
%sc x

-> Synth('mySynth' : 1006)


In [16]:
%sc x.set(\freq, 1800, \amp, 0.01, \sFreq, 30) // parameter update

-> Synth('mySynth' : 1006)


In [17]:
%sc x.set(\freq, 2000, \amp, 0.1, \sFreq, 20) // parameter update

-> Synth('mySynth' : 1006)


In [18]:
%sc s.scope

-> a Stethoscope


Let's understand what it does:
- Saw is a negative ramp, here from 5 to -5
- Clip clips default by 0 and 1
- check scope to see waveform

In [19]:
%sc x.free // termininate and deletes an instance 

-> Synth('mySynth' : 1006)


In [20]:
%sc {Saw.kr(10, 5)}.plot(0.3)

-> a Plotter


In [19]:
%sc Saw.help

-> Saw


different programming techniques in SC
(based on <http://swiki.hfbk-hamburg.de:8888/MusicTechnology/571 by jrh)

## Interaction Styles

### function-style low-level via OSC

In [127]:
%sc Server.help

-> Server


In [128]:
%scv s.sendMsg("/s_new", "mySynth", 1020, 0, 1, "freq", 300, "amp", 0.1);

-> sc3nb_remote


In [129]:
%%sc
s.sendMsg("/n_set", 1020, "freq", 400, "sFreq", 1.5, "amp", 0.1);

-> sc3nb_remote


In [130]:
%sc s.sendMsg("/n_free", 1020);

-> sc3nb_remote


In [24]:
%scs Server.help // see Server Command Reference

* (+) no Overhead (fast!)
* (+) ideal for Granular synthesis
* (–) no OO Methods useable, really low-level interface
* (–) no dynamic allocation of synths

### function-style via Synth class

In [135]:
%sc Synth.help

-> Synth


In [131]:
%sc x = Synth(\mySynth)

-> Synth('mySynth' : 1007)


In [132]:
%sc x.set(\freq, 800, \amp, 0.05, \sFreq, 802)

-> Synth('mySynth' : 1007)


In [133]:
%sc x.free

-> Synth('mySynth' : 1007)


In [134]:
%scv x.nodeID

-> 1007


In [136]:
%%scv 
~xarr = 5.collect{
    Synth(\mySynth, [\freq, 500.rand, \amp, 0.06, \sFreq, 10.0.rand])}

-> [ Synth('mySynth' : 1008), Synth('mySynth' : 1009), Synth('mySynth' : 1010), Synth('mySynth' : 1011), Synth('mySynth' : 1012) ]


In [145]:
%scv ~xarr[4].free

-> Synth('mySynth' : 1012)


In [146]:
%scv ~xarr.do{|r| r.nodeID.postln}

1008
1009
1010
1011
1012
-> [ Synth('mySynth' : 1008), Synth('mySynth' : 1009), Synth('mySynth' : 1010), Synth('mySynth' : 1011), Synth('mySynth' : 1012) ]


In [32]:
%sc ~xarr[4].set(\freq, 1000, \sFreq, 50)

-> Synth('mySynth' : 1012)


* (+) OO functionality 
* (+) hidden Interface between sclang und scsynth
* (+) better readability of code 
* (+) ID Management hidden 
* (–) Overhead (opposed to message-style invocation)

In [33]:
%sc ~xarr.do{|i| i.free}

-> [ Synth('mySynth' : 1008), Synth('mySynth' : 1009), Synth('mySynth' : 1010), Synth('mySynth' : 1011), Synth('mySynth' : 1012) ]


### function-style with implicit Synth creation

In [151]:
%sc {SinOsc.ar(300, mul: 0.05)}.play

-> Synth('temp__12' : 1028)


In [148]:
%%scv
x = {
    SinOsc.ar(400, 0, 0.2) 
    * SinOsc.ar(freq: SinOsc.ar(0.2).range(20)!2)
}.play

-> Synth('temp__1' : 1015)


In [36]:
%sc x.free

-> Synth('temp__1' : 1014)


* (+) fast to write 
* (-) lot of overhead:
    * all Synths are created out of temporary synthdefs 

### JITLib-style

In [37]:
%scv p = ProxySpace.push(s);

-> ProxySpace ()


In [38]:
%scv ~x.play

-> NodeProxy.audio(sc3nb_remote, 2)


In [39]:
%scv ~x = {SinOsc.ar(~lfo.ar * 106, mul: 0.05)}

-> a Function


In [40]:
%scv ~lfo = {Pulse.ar(MouseY.kr(1,10), mul: MouseX.kr(1,10))!2}

-> a Function


now components can be replaced and changed while the sound is continuously rendered:

In [41]:
%scv ~x = {Blip.ar(~lfo.ar * 106, mul: 0.2, numharm: 2)}

-> a Function


In [42]:
%sc ~lfo = {SinOsc.ar(6)!2}

-> a Function


In [43]:
%scv ~x.free

-> NodeProxy.audio(sc3nb_remote, 2)


* (+) rapid prototyping
* (+) live coding
* (+) little code overhead
* (+) maintains settings independent from the content 
* (+) lazy environment
* (–) may encourages people to write inefficient code
* (–) reusability is difficult to implement

In [152]:
%%sc
SynthDef(\braap, {|freq = 70, out = 0, sustain = 0.3, amp = 0.1| 
    var line;
    
    line = EnvGen.kr(Env.perc(sustain, 0.01, 8, -4), doneAction: 2); 
    
    Out.ar(out, (SinOsc.ar(freq) * line).softclip!2 * amp)
}).store

-> a SynthDef


In [153]:
%scs (instrument: \braap, freq: 130).play // event

In [154]:
%scs (instrument: \braap, freq: 230, sustain: 2).play // event

In [155]:
%scs (instrument: \braap, freq: 430, sustain: 0.2).play // event

In [156]:
%scs (instrument: \braap, freq: [54, 575]).play // event

In [158]:
%scs (instrument: \braap, midinote: 69).play // event

In [159]:
%scs (instrument: \braap, midinote: 35).play // event

In [161]:
%scs Pseq([(instrument: \braap, midinote: 70, dur: 0.1)], 8).play

In [162]:
%%scs
q = q ? ();
q.l.stop;
q.l = Pbind(
    \instrument, \braap,
    \note, Pseq([12, 0, 12, 24], inf) + Pseq(([5, 3, -2, 1]!16).flop.flat, inf),
    \octave, [1, 3, 2, 4],
    \root, 12,
    \dur, Pseq([0.5, 0.1].normalizeSum * 0.25, inf),
    \amp, 0.1*[0.1, 0.08, Pseq([Pgeom(0, 0.01, 50)], inf), 0.1]
).play

In [163]:
%scv q.l.stop

-> an EventStreamPlayer


## Typical Elements of a synthesizer 

Tour of UGens: <https://doc.sccode.org/Guides/Tour_of_UGens.html>

In [54]:
%sc s.scope; FreqScope(400)

-> FreqScope


In [55]:
%sc s.makeGui

-> sc3nb_remote


In [56]:
%sc s.plotTree

-> sc3nb_remote


In [57]:
%sc x.inspect

-> an ObjectInspector


### Oscillators 

In [164]:
%sc { SinOsc.ar(800, mul: 0.1, add: 0)!2}.play

-> Synth('temp__13' : 1522)


In [59]:
%sc Pulse.help

-> Pulse


In [166]:
%sc s.scope

-> a Stethoscope


In [165]:
%sc { Pulse.ar(200, MouseX.kr(0.01, 1), 0.1)!2 }.play

-> Synth('temp__14' : 1524)


In [167]:
%sc x = { LFPulse.ar(200, 0, MouseX.kr(0.01, 1), 0.05)!2 }.play

-> Synth('temp__15' : 1528)


In [168]:
%sc { Impulse.ar(MouseX.kr(1,4000))!2 }.play

-> Synth('temp__16' : 1530)


In [169]:
%sc { VarSaw.ar(600,0,MouseX.kr(0,1),0.1)!2 }.play

-> Synth('temp__17' : 1532)


In [170]:
x = %sc { 0.1* Blip.ar(MouseY.kr(200,5000), MouseX.kr(1,100), 0.5)!2}.play

-> Synth('temp__18' : 1534)


In [171]:
from ipywidgets import interactive

In [172]:
%scg x = { | freq=200, num=1, amp=0.2 | Blip.ar(freq, num, amp)!2}.play

-> Synth('temp__19' : 1536)


()

In [173]:
%sc Blip.help

-> Blip


In [174]:
def setpars(num=1, freq=100, amp=0.2):
    sc.lang.cmd(r"x.set(\num, ^p1)", pyvars={"p1": num}, verbose=False)
    sc.lang.cmd(r"x.set(\freq, ^p1)", pyvars={"p1": freq}, verbose=False)
    sc.lang.cmd(r"x.set(\amp, ^p1)", pyvars={"p1": amp}, verbose=False)
interactive(setpars, num=(1, 10, 1), freq=(100, 300, 1), amp=(0.01, 0.8, 0.02))

interactive(children=(IntSlider(value=1, description='num', max=10, min=1), IntSlider(value=100, description='…

In [175]:
%sc x.free

-> Synth('temp__19' : 1536)


### Spatialization, e.g. Pan2, PanAz 

In [3]:
%sc Pan2.help

CDoc: Indexed 2230 documents in 2.41 seconds
-> Pan2
sc3> Pan2.help;


In [176]:
%sc { Pan2.ar( WhiteNoise.ar(0.3), MouseX.kr(-1, 1), 0.2 ) }.play

-> Synth('temp__20' : 1537)


In [15]:
%sc { Pan2.ar( SinOsc.ar(440), pos: 0, level: 0.5 )}.play

-> Synth('temp__7' : 201327601)


In [16]:
%sc { Pan2.ar( SinOsc.ar(440), pos: -1, level: 0.5 )}.play

-> Synth('temp__8' : 201327602)


In [17]:
%sc { Pan2.ar( SinOsc.ar(440), pos: MouseX.kr(-1,1), level: 0.5 )}.play

-> Synth('temp__9' : 201327603)


### Envelopes  

In [177]:
# %sc {EnvGen.kr(Env.new([0,1,1,0], [2,2,0.1]))}.plot(4.1)
# %sc {SinOsc.ar(10)}.plot(0.2)
%sc EnvGen.help

-> EnvGen


In [178]:
%sc Env.new(levels: [0, 1, 0.9, 0], times: [0.1, 0.5, 1], curve: [5, 0, -5]).plot;

-> a Plotter


In [14]:
%%sc
{ 
    EnvGen.kr(Env.new([0,1,1,0], [1,1,0.1]), 1.0, doneAction: 2) 
    * 
    SinOsc.ar(440, 0, 0.1)!2
}.plot(2.1)

-> a Plotter


In [20]:
%%sc 
{ 
    var freqenv = EnvGen.kr(Env.new([0,1,1,0], [2,2,0.1]), 1.0, 2.0, doneAction: 2);  
    SinOsc.ar(440*freqenv, 0, 0.3)!2
}.play

-> Synth('temp__11' : 201327606)


### putting things together

In [70]:
%sc EnvGen.help

-> EnvGen


In [179]:
%%scv
SynthDef("mysyn", { | freq=400, amp=0.5, att=0.1, dur=2, rel=0.5, pan=0 |
    var sig, env; 
    sig = SinOsc.ar(freq);
    env = EnvGen.kr(Env.new([0,1,1,0], [att, dur-att-rel, rel]), 
                    levelScale: amp, doneAction: 2);
    Out.ar(0, Pan2.ar(sig, pan, env))
}).send(s)

-> a SynthDef


In [184]:
%sc x = Synth.new(\mysyn, [\pan, -1, \freq, 1500, \dur, 0.2, \att, 0.001, \pan, -1])

-> Synth('mysyn' : 1543)


In [194]:
%%sc 
{ 150.do{ 
    Synth.new(\mysyn, [
        \freq, (40.rand+60).midicps, 
        \amp, 0.05, 
        \att, 0.01.rand, 
        \dur, 1.rand, 
        \rel, 1.5.rand, 
        \pan, 2.rand-1]);
    0.05.rand.wait;
}}.fork

-> a Routine
[sclang | reached EOF ]


### Noise and Filters

In [74]:
%%sc
Routine({
    x = { WhiteNoise.ar(0.6) }.play; 1.wait; x.free;
    x = { PinkNoise.ar(0.6) }.play;  1.wait; x.free;
    x = { BrownNoise.ar(0.6) }.play; 1.wait; x.free;
}).play

-> a Routine


In [75]:
%scs s.scope

In [76]:
%sc { Dust2.ar(MouseX.kr(1,10000,1), 1) !2}.scope(1, zoom:4);

-> Synth('temp__12' : 1231)


In [77]:
%sc { Klank.ar(`[2*[600, 671, 1353, 4723], nil, 0.5*[1.5, 0.5, 0.3, 0.2]], Impulse.ar(1, 0, 0.2))!2 }.play;

-> Synth('temp__13' : 1235)


In [78]:
%sc { Klank.ar(`[{5000.rand}!10, nil, {1.0.rand}!10], Impulse.ar(1, 0, 0.2))!2 }.play;

-> Synth('temp__14' : 1238)


In [79]:
%%sc 
{ 
    DynKlank.ar(`[[200, 671, 1153, 1723], nil, 
                  MouseX.kr(0.01, 5)*[1, 1, 1, 1]], 
                PinkNoise.ar(0.007))!2
}.play

-> Synth('temp__15' : 1241)


In [80]:
%%sc 
{ Klank.ar(`[ {exprand(200, 8000)}.dup(12), nil, nil ], 
           PinkNoise.ar(0.007))!2 }.play

-> Synth('temp__16' : 1244)


## Tasks

### Task 1:

Create a synth that sounds like a water drop 
* linear increase of pitch of a rather sinusoidal oscillator
* amplitude should decay exponentially

(optional:) 
Create a rain simulator where you can control the stochastic density of rain drops
* and their chirp speed mean and spread 
* and amplitude decay speed (mean and spread)
* and explore what different dripping collages this can create

In [81]:
# cue: For an exponential decay (or rise), XLine is a helpful UGen
%sc {XLine.kr(start: 1, end: 0.1, dur: 0.5)}.plot(0.5)

-> a Plotter


In [82]:
%sc XLine.help

-> XLine


In [83]:
%sc {SinOsc.ar(freq: XLine.kr(400, 1000, 0.3), mul: XLine.kr(1, 0.01, 0.2, doneAction: 2))}.play

-> Synth('temp__17' : 1248)


In [54]:
%sc s.scope

-> a Stethoscope


here a solution to the problem: but please try yourself before reading the following code

#### Solution

In [84]:
%%sc 
SynthDef(\drop, { | f0=100, f1=500, dur=0.5, amp=0.1, pan=0 |
    var sig, fenv, env, s2;
    fenv = Line.kr(f0, f1, dur, doneAction: 2);
    env = XLine.kr(1, 0.001, dur);
    sig = SinOsc.ar(fenv, mul: env);
    s2 = Pan2.ar(sig, pan, amp);
    Out.ar([0,1], s2);             
}).add()

-> a SynthDef


In [85]:
%sc Synth(\drop, [\f0, 600, \f1, 1200, \dur, 0.05, \amp, 0.1])

-> Synth('drop' : 1249)


In [86]:
%%sc 
{
    30.do{
        var f0 = 300 + (500.rand);
        Synth(\drop, [\f0, f0, 
                      \f1, f0 + (600.rand), 
                      \dur, 0.2.rand,
                      \amp, 0.1.rand,
                      \pan, 2.0.rand -1] );
    0.3.rand.wait;
    }
}.fork

-> a Routine


here another possible solution

In [87]:
%%scv
SynthDef(\drop, { | freq=400, dp=600, amp=0.05, dur=2, pan=0 |
    var sig, env, fch;
    fch = XLine.kr(freq, freq+dp, dur);
    sig = SinOsc.ar(fch);
    env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
    Out.ar(0, Pan2.ar(sig, pan, env*amp))
}).send(s)

-> a SynthDef


In [88]:
%scv Synth.new(\drop, ["freq", 500, "dp", 1200, "dur", 0.1]);

-> Synth('drop' : 1280)


In [89]:
%%scv
q = q ? ();
q.numChans = 2

-> ( 'l': an EventStreamPlayer, 'numChans': 2 )


In [90]:
%%scv 
SynthDef(\FreeVerb2x2, {|outbus, mix = 0.25, room = 0.15, damp = 0.5, amp = 1.0|
    var signal;
    q.numChans.do{ |i|
        signal = In.ar(outbus+i, 1);
        ReplaceOut.ar(outbus+i, FreeVerb.ar( signal, mix, room, damp, amp)); 
    }
}).send(s);

-> a SynthDef
[sclang | reached EOF ]
[sclang | reached EOF ]
[scsynth | reached EOF ]


here a physically more correct solution, engineered after the [ICAD paper by K. v.d. Doel](http://www.icad.org/websiteV2.0/Conferences/ICAD2004/papers/van_den_doel.pdf)

In [64]:
%%scv
SynthDef(\drop, { |out=0, f0 = 800, u = 10, xi = 0.5, pos=0.5, wid=2, orient=0, gain=0|
    var damp = (0.043*f0) + (0.0014*(f0**0.67));
    var dur  = damp.reciprocal * 3;
    var df   = xi * damp;
    var freq = EnvGen.ar(Env( [f0, f0+df], [dur]), doneAction: 2);
    var amp  = ((3.0/freq)**1)*u;    // original exp 1.5 -> too strong decrease with f0 
    var env  = EnvGen.ar(Env.perc(dur*0.001, dur*0.999, 1, -4));
    var sig = SinOsc.ar(freq, mul: gain.dbamp*amp);
    Out.ar(out, PanAz.ar(q.numChans, sig, pos, env, wid, orient))
}).send(s);

-> a SynthDef


Reverberation is helpful and makes the sound more natural

In [66]:
%%scv 
z = Synth(\FreeVerb2x2, [\outbus, 0], addAction:\addToTail);
//set reverb settings 
z.set(\room, 1);
z.set(\mix, 0.15);
z.set(\damp, 0.8);

-> Synth('FreeVerb2x2' : 1339)


In [77]:
%%scv 
q.glevel = (-0);
q.dt = 1.0;

-> ( 'numChans': 2, 'glevel': 0, 'dt': 1.0 )
[scsynth | reached EOF ]


now let's listen: it sounds like sounds in a cave

In [76]:
%%scv
{100.do{
    var f0 = 1.0.rand.linlin(0, 1, 400, 1000);
    var u = 1.0.rand.linlin(0, 1, 1, 20);
    var xi = 1.0.rand.linlin(0, 1, 0.1, 20);
    var pos = 1.0.rand.linlin(0, 1, -1, 1);
    var gain = 1.0.rand.linlin(0, 1, -6, 20) + q.glevel;
    Synth.new(\drop, [\out, 0, \pos, pos, \u, u, \f0, f0, \xi, xi, \gain, gain]); 
    q.dt.rand.wait;
}
}.fork

-> a Routine


In [70]:
# increase level by 12 dB if needed
%scv q.glevel = q.glevel + 12

-> ( 'numChans': 2, 'glevel': 12, 'dt': 1.0 )


In [73]:
%scv q.glevel = q.glevel - 12

-> ( 'numChans': 2, 'glevel': -12, 'dt': 1.0 )


or change maximum random waiting time between drops

In [75]:
%sc q.dt = 0.1

-> ( 'numChans': 2, 'glevel': -12, 'dt': 0.1 )


In [136]:
%sc s.freeAll

-> sc3nb_remote


In [21]:
sc.exit()

Quitting SCServer... Done.
Exiting sclang... Done.


### Task 2:

Create a synth with a sine oscillator whose frequency oscillates itself around its center value with a sinusoidal function, i.e. 
$$ f = f_0 + df*\sin(2\pi f_v t) ) $$
and whose amplitude oscillates likewise with another frequency $f_a$.
Finally the location should oscillates sinusoidal with a third frequency $f_p$

#### Solution (Don't look at it..., try yourself)

In [87]:
%%sc 
x = { | ff=4, dff=20, f0=200, fa=4, dfa=0.2, fo=2| 
    Pan2.ar(
        SinOsc.ar(
            SinOsc.ar(ff, mul: dff, add: f0), 
            SinOsc.ar(fa, mul: dfa, add: dfa)
        ), 
        SinOsc.ar(fo, mul: 1)
    )
}.play

In [88]:
%sc x.set(\f0, 300)

-> Synth('temp__28' : 1017)


In [89]:
%sc x.set(\ff, 450)

-> Synth('temp__28' : 1017)


In [90]:
%sc x.set(\dff, 200)

-> Synth('temp__28' : 1017)


In [91]:
%sc x.set(\fo, 0.1)

-> Synth('temp__28' : 1017)


In [92]:
%sc x.set(\fa, 2)

-> Synth('temp__28' : 1017)


In [93]:
%sc x.set(\dfa, 1)

-> Synth('temp__28' : 1017)


In [94]:
%%scv 
q = q ? ();

SynthDef(\mySynth, { |freq=200, amp=1, sFreq=1| 
    Out.ar(0, SinOsc.ar(freq, mul:amp) * Clip.ar(Saw.ar(sFreq, 5)));
}).send(s);

q.soundon = { | q, steps = 10 |
    q.synarr = steps.collect{ | i |
        Synth(\mySynth, [\freq, 400.0.rand + 200, \amp, 0.1, \sFreq, 15.0.rand]);
    };
}

-> ( 'l': an EventStreamPlayer, 'soundon': a Function )


In [95]:
%%scv 
q.soundstop = {
    q.synarr.do{ |p| p.free};
}

-> ( 'l': an EventStreamPlayer, 'soundstop': a Function, 'soundon': a Function )


In [96]:
%scv q.soundon(10)

-> ( 'l': an EventStreamPlayer, 'synarr': [ Synth('mySynth' : 1018), Synth('mySynth' : 1019), Synth('mySynth' : 1020), Synth('mySynth' : 1021), Synth('mySynth' : 1022), Synth('mySynth' : 1023), Synth('mySynth' : 1024), Synth('mySynth' : 1025), Synth('mySynth' : 1026), Synth('mySynth' : 1027) ], 'soundstop': a Function, 'soundon': a Function )


In [97]:
%sc q.soundstop()

-> [ Synth('mySynth' : 1018), Synth('mySynth' : 1019), Synth('mySynth' : 1020), Synth('mySynth' : 1021), Synth('mySynth' : 1022), Synth('mySynth' : 1023), Synth('mySynth' : 1024), Synth('mySynth' : 1025), Synth('mySynth' : 1026), Synth('mySynth' : 1027) ]


In [98]:
%sc { SinOsc.ar(1000 + (2 * 44100))}.plot(0.001) // same signal for 2, 3, 4... at sr=44100